In [1]:
using System;
using System.Collections.Generic;
using System.Linq;

public class AdaptiveTicker
{
    private decimal Base { get; }
    private List<int> Mantissas { get; }
    private decimal MinInterval { get; }
    private decimal MaxInterval { get; }
    private int DesiredNumTicks { get; }
    private int NumMinorTicks { get; }

    public AdaptiveTicker(decimal @base = 10.0m, List<int> mantissas = null, decimal minInterval = 0.0m, decimal? maxInterval = null)
    {
        Base = @base;
        Mantissas = mantissas ?? new List<int> { 1, 2, 5 };
        MinInterval = minInterval;
        MaxInterval = maxInterval ?? decimal.MaxValue;
        DesiredNumTicks = 6;
        NumMinorTicks = 5;
    }

    public Dictionary<string, List<decimal>> GetTicks(decimal dataLow, decimal dataHigh)
    {
        return GetTicksNoDefaults(dataLow, dataHigh, DesiredNumTicks);
    }

    public Dictionary<string, List<decimal>> GetTicksNoDefaults(decimal dataLow, decimal dataHigh, int desiredNTicks)
    {
        decimal interval = GetInterval(dataLow, dataHigh, desiredNTicks);
        int startFactor = (int)Math.Floor(dataLow / interval);
        int endFactor = (int)Math.Ceiling(dataHigh / interval);

        var ticks = Enumerable.Range(startFactor, endFactor - startFactor + 1)
                              .Select(factor => factor * interval)
                              .Where(tick => dataLow <= tick && tick <= dataHigh)
                              .ToList();

        var minorTicks = new List<decimal>();
        if (NumMinorTicks > 0 && ticks.Any())
        {
            decimal minorInterval = interval / NumMinorTicks;
            minorTicks = ticks.SelectMany(tick => Enumerable.Range(0, NumMinorTicks)
                                                            .Select(x => tick + x * minorInterval)
                                                            .Where(mt => dataLow <= mt && mt <= dataHigh))
                              .ToList();
        }

        return new Dictionary<string, List<decimal>>
        {
            ["major"] = ticks,
            ["minor"] = minorTicks
        };
    }

    public decimal GetMinInterval() => MinInterval;

    public decimal GetMaxInterval() => MaxInterval;

    public List<decimal> ExtendedMantissas()
    {
        decimal prefixMantissa = Mantissas.Last() / Base;
        decimal suffixMantissa = Mantissas.First() * Base;
        return new[] { prefixMantissa }.Concat(Mantissas.Select(m => (decimal)m)).Concat(new[] { suffixMantissa }).ToList();
    }

    public decimal BaseFactor() => MinInterval == 0.0m ? 1.0m : MinInterval;

    public decimal GetInterval(decimal dataLow, decimal dataHigh, int desiredNTicks)
    {
        decimal dataRange = dataHigh - dataLow;
        decimal idealInterval = GetIdealInterval(dataLow, dataHigh, desiredNTicks);

        int intervalExponent = (int)Math.Floor(Math.Log((double)(idealInterval / BaseFactor()), (double)Base));
        decimal idealMagnitude = (decimal)Math.Pow((double)Base, intervalExponent) * BaseFactor();

        var candidateMantissas = ExtendedMantissas();

        var errors = candidateMantissas.Select(mantissa => Math.Abs(desiredNTicks - (dataRange / (mantissa * idealMagnitude)))).ToList();
        decimal bestMantissa = candidateMantissas[ArgMin(errors)];
        decimal interval = bestMantissa * idealMagnitude;

        return Math.Max(Math.Min(interval, GetMaxInterval()), GetMinInterval());
    }

    public decimal GetIdealInterval(decimal dataLow, decimal dataHigh, int desiredNTicks)
    {
        // Placeholder for method to calculate ideal interval based on data and desired ticks
        return (dataHigh - dataLow) / desiredNTicks;
    }

    public int ArgMin(List<decimal> values)
    {
        return values.IndexOf(values.Min());
    }
}


In [4]:
var ticker = new AdaptiveTicker();

In [5]:
ticker.GetTicks(1.0m, 2.0m)

key,value
major,"[ 1.00, 1.20, 1.40, 1.60, 1.80, 2.00 ]"
minor,"[ 1.00, 1.04, 1.08, 1.12, 1.16, 1.20, 1.24, 1.28, 1.32, 1.36, 1.40, 1.44, 1.48, 1.52, 1.56, 1.60, 1.64, 1.68, 1.72, 1.76 ... (6 more) ]"
